In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping, shape
from imp import reload
from numpy import mean
from planet_utils import search, download
from numpy.random import randint, choice
import random
import folium
import json
import requests
import os
import smart_open
from retrying import retry
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Circle
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool 

plt.rcParams['figure.figsize'] = (20,20)
%matplotlib inline

#print(os.environ["PL_API_KEY"])
NUM_RANDOM_DATES = 10
NUM_RANDOM_LOCATIONS = 20

image_utils: found API key


# API Search Candidate Selection Protocol
The goal of this notebook is to develop the pathway from a set of single-measurement points to a set of cropped PlanetScope imagery for a given date band. 

## Extract 2017 Measurement Locations

In [13]:
snowdata = pd.read_csv("../data/snow_summary_all_2009_2017_locs.csv", 
                       parse_dates = ["snow_appearance_date", "snow_disappearance_date", 
                                      "date_min", "date_max"])
snowdata = snowdata[snowdata.year >= 2017]
snowdata['geometry'] = [Point(xy) for xy in zip(snowdata.longitude, snowdata.latitude)]
snowdata = gpd.GeoDataFrame(snowdata)
snowdata.crs = {'init' : 'epsg:4326'}


In [14]:
locations = snowdata.dropna(subset=["longitude", 'latitude']).drop_duplicates("Location")



In [15]:
locations = locations.loc[choice(locations.index, NUM_RANDOM_LOCATIONS, replace=False)]


In [16]:
len(locations)

20

## Add bounding boxes

In [17]:
boxes = locations[['Location', 'geometry']].copy()
boxes.geometry = [g.buffer(0.005, cap_style=3) for g in boxes.geometry]

## Search

In [18]:
boxes.sample(1)

,Location,geometry
1882,TO11-S5,"POLYGON ((-121.8822 46.99967, -121.8822 46.989..."


In [21]:
reload(search)
dates = locations[['Location', "snow_appearance_date", "snow_disappearance_date"]]
searcher = search.Search(boxes, dates, dry=False,
                         key='Location', start_col='snow_appearance_date',
                         end_col="snow_disappearance_date")
results = searcher.query()

Querying Planet API: 100%|██████████| 20/20 [01:09<00:00,  3.46s/searches]


## Parse Results
Choose `NUM_RANDOM_DATES` dates from results for each loc

In [22]:
loc_img_ids = {}
for group in results.groupby('loc_id'):
    if (len(group[1]) >= NUM_RANDOM_DATES):
        loc_img_ids[group[0]] = list(set(choice(group[1].id.values, NUM_RANDOM_DATES, replace=False)))
    else:
        loc_img_ids[group[0]] = list(set(group[1].id.values))



In [25]:
CLIP_API_URL = "https://api.planet.com/compute/ops/clips/v1/"
IMAGEDIR = "s3://planet-snowcover/tmp/"
PL_API_KEY = os.environ["PL_API_KEY"]

In [35]:
def clip_request_and_download(loc, image):

    @retry(wait_fixed=5000)
    def _check_clip_op(id):
        r = requests.get("{_base}/{id}".format(_base = CLIP_API_URL, id=id), auth=(PL_API_KEY, ""))
        if r.json()['state'] != "succeeded":
            print("...waiting")
            raise Exception("Not Yet")
        else:
            print("response found.")
            return(r.json())
    
    geom = boxes.loc[loc].geometry
        
    payload = {
        "aoi" : mapping(geom),
        "targets" : [{
            "item_id" : image, 
            "item_type" : "PSScene4Band", 
            "asset_type" : 'analytic'
        }]
    }

    r = requests.post(CLIP_API_URL, auth=(PL_API_KEY, ""), json=payload)
    print(r.json())
    
    response = _check_clip_op(r.json()['id'])

    image_url = response['_links']['results'][0]
    
    local_filename = os.path.join(IMAGEDIR, "{loc}_{img}.zip".format(loc=loc, img=image))

    r = requests.get(image_url, stream=True, auth=(PL_API_KEY, ""))
    with smart_open.smart_open(local_filename, 'wb', profile_name="buckleylab") as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename



In [27]:
len(loc_img_ids)

20

In [31]:
list(loc_img_ids.items())[0]

(1216,
 ['20170105_174052_0c45',
  '20170124_181608_0e20',
  '20170403_181107_1012',
  '20161230_193630_0d06',
  '20170115_181452_0e19',
  '20161117_181314_0e26',
  '20170103_180347_0c41',
  '20170305_181650_0e19',
  '20170403_181108_1012',
  '20170202_181629_0e0f'])

In [37]:
reload(download)
files = {}
for loc_id, img_ids in [list(loc_img_ids.items())[0]]:
    box = boxes.loc[loc_id].geometry
    dl = download.CroppedDownload(loc_id, box, img_ids, IMAGEDIR)
    files[loc_id] = dl.run()


Starting download for image 20170403_181107_1012
Starting download for image 20170124_181608_0e20
Starting download for image 20161230_193630_0d06
Starting download for image 20170105_174052_0c45
Starting download for image 20170115_181452_0e19
starting...{'aoi': {'type': 'Polygon', 'coordinates': [[[-121.53224, 46.92512000000001], [-121.53224, 46.91512], [-121.54223999999999, 46.91512], [-121.54223999999999, 46.92512000000001], [-121.53224, 46.92512000000001]]]}, 'id': '0bccd108-94e0-4a71-b347-18349c6bb729', 'state': 'running', 'created_on': '2018-12-17T22:42:14.451Z', '_links': {'results': None, '_self': 'https://api.planet.com/compute/ops/clips/v1/0bccd108-94e0-4a71-b347-18349c6bb729'}, 'last_modified': '2018-12-17T22:42:14.451Z', 'targets': [{'item_id': '20170403_181107_1012', 'item_type': 'PSScene4Band', 'asset_type': 'analytic'}]}
starting...{'aoi': {'type': 'Polygon', 'coordinates': [[[-121.53224, 46.92512000000001], [-121.53224, 46.91512], [-121.54223999999999, 46.91512], [-121

In [38]:
files

{1216: ['s3://planet-snowcover/tmp/1216_20170105_174052_0c45.zip',
  's3://planet-snowcover/tmp/1216_20170124_181608_0e20.zip',
  's3://planet-snowcover/tmp/1216_20170403_181107_1012.zip',
  's3://planet-snowcover/tmp/1216_20161230_193630_0d06.zip',
  's3://planet-snowcover/tmp/1216_20170115_181452_0e19.zip',
  's3://planet-snowcover/tmp/1216_20161117_181314_0e26.zip',
  's3://planet-snowcover/tmp/1216_20170103_180347_0c41.zip',
  's3://planet-snowcover/tmp/1216_20170305_181650_0e19.zip',
  's3://planet-snowcover/tmp/1216_20170403_181108_1012.zip',
  's3://planet-snowcover/tmp/1216_20170202_181629_0e0f.zip']}